# Week 1: Geometric Foundations - Integration Demo

This notebook demonstrates the end-to-end process of applying geometric corrections to an image, combining concepts learned in Week 1:
1.  Loading camera calibration data (intrinsics, distortion, undistortion maps).
2.  Loading a sample image (e.g., from an ESP32-CAM).
3.  Applying lens undistortion using the calibration data.
4.  Defining a region of interest (ROI) in the undistorted image.
5.  Calculating and applying a perspective warp (homography) to obtain a rectified, top-down view of the ROI.
6.  Visualizing the results at each step.

In [1]:
# Import necessary libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

## 1. Configuration and Setup

Define file paths, output directory, and the points for the perspective warp.
**Important:** Adjust `UNDISTORTED_SRC_POINTS` based on your specific test image after viewing the undistorted version.


In [ ]:
# --- Configuration ---
CALIBRATION_FILE = 'camera_calibration_data.npz'
TEST_IMAGE_FILE = 'esp32_test_image.jpg' # Image captured by ESP32-CAM or similar
OUTPUT_DIR = 'output_week1_notebook' # Directory to save results from notebook

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Define the Warp Transformation ---
# Source points in the *undistorted* image (TL, TR, BR, BL order)
# *** Adjust these points based on your TEST_IMAGE_FILE after undistortion! ***
UNDISTORTED_SRC_POINTS = np.float32([
    [150, 100],  # Top-Left corner of ROI in undistorted image
    [450, 110],  # Top-Right corner
    [500, 350],  # Bottom-Right corner
    [100, 340]   # Bottom-Left corner
])

# Define the desired output rectangle size and shape for the warped region
OUTPUT_WIDTH = 300
OUTPUT_HEIGHT = 250
TARGET_DST_POINTS = np.float32([
    [0, 0],                     # Target Top-Left
    [OUTPUT_WIDTH - 1, 0],      # Target Top-Right
    [OUTPUT_WIDTH - 1, OUTPUT_HEIGHT - 1], # Target Bottom-Right
    [0, OUTPUT_HEIGHT - 1]      # Target Bottom-Left
])

# Function to display images using Matplotlib
def display_image(title, img_bgr):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

def display_comparison(title1, img1_bgr, title2, img2_bgr):
    img1_rgb = cv2.cvtColor(img1_bgr, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2_bgr, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img1_rgb)
    plt.title(title1)
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(img2_rgb)
    plt.title(title2)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
print("Configuration set.")


## 2. Load Calibration Data

Load the pre-computed camera matrix, distortion coefficients, and undistortion maps saved from the calibration process.


In [ ]:
# --- Load Calibration Data ---
try:
    with np.load(CALIBRATION_FILE) as data:
        mtx = data['mtx']
        dist = data['dist']
        new_camera_mtx = data['new_camera_mtx']
        roi = data['roi']
        map1 = data['map1']
        map2 = data['map2']
        img_size = tuple(data['img_size'])
    print(f"Calibration data loaded successfully from '{CALIBRATION_FILE}'")
    print("Camera Matrix (mtx):\n", mtx)
    print("Distortion Coefficients (dist):\n", dist)
    print("Image Size from Calibration:", img_size)
except Exception as e:
    print(f"Error loading calibration data from '{CALIBRATION_FILE}': {e}")
    # Stop execution if calibration data is missing
    raise

## 3. Load Test Image

Load the sample image that we want to apply the corrections to.


In [ ]:
# --- Load Test Image ---
original_img = cv2.imread(TEST_IMAGE_FILE)

if original_img is None:
    print(f"Error: Could not load test image from '{TEST_IMAGE_FILE}'")
    # Stop execution if image is missing
    raise FileNotFoundError(f"Test image not found: {TEST_IMAGE_FILE}")
else:
    print(f"Test image '{TEST_IMAGE_FILE}' loaded successfully.")
    # Verify image size (optional but recommended)
    h, w = original_img.shape[:2]
    if (w, h) != img_size:
        print(f"Warning: Test image size {(w,h)} differs from calibration image size {img_size}. Resizing...")
        original_img = cv2.resize(original_img, img_size)
        h, w = original_img.shape[:2] # Update dimensions
        print(f"Resized test image dimensions: {(w, h)}")
        
    # Display the original image
    display_image('1. Original Test Image', original_img)
    cv2.imwrite(os.path.join(OUTPUT_DIR, '1_original.jpg'), original_img)


## 4. Apply Lens Undistortion

Use the pre-computed `map1` and `map2` with `cv2.remap` to efficiently remove lens distortion from the test image.


In [ ]:
# --- Apply Undistortion ---
print("Applying lens undistortion using cv2.remap...")
undistorted_img = cv2.remap(original_img, map1, map2, interpolation=cv2.INTER_LINEAR)
print("Undistortion applied.")

# Optional: Crop using ROI
x, y, w_roi, h_roi = roi
undistorted_cropped = undistorted_img[y:y+h_roi, x:x+w_roi]

# Display comparison
display_comparison('1. Original Image', original_img, '2. Undistorted Image', undistorted_img)
cv2.imwrite(os.path.join(OUTPUT_DIR, '2_undistorted.jpg'), undistorted_img)
cv2.imwrite(os.path.join(OUTPUT_DIR, '3_undistorted_cropped.jpg'), undistorted_cropped) # Save cropped too

## 5. Apply Perspective Warp

Calculate the homography matrix that maps the `UNDISTORTED_SRC_POINTS` (defined in the undistorted image) to the rectangular `TARGET_DST_POINTS`. Then, apply this transformation to the undistorted image using `cv2.warpPerspective`.

In [ ]:
# --- Calculate and Apply Perspective Warp ---
print("Calculating Homography matrix...")
# Use getPerspectiveTransform as we have exactly 4 points
homography_matrix = cv2.getPerspectiveTransform(UNDISTORTED_SRC_POINTS, TARGET_DST_POINTS)
# Alternative: findHomography (more robust if points are noisy or > 4)
# homography_matrix, status = cv2.findHomography(UNDISTORTED_SRC_POINTS, TARGET_DST_POINTS) 

if homography_matrix is None:
    print("Error: Could not compute homography matrix.")
    # Stop execution
    raise ValueError("Homography computation failed.")
else:
    print("Homography Matrix:\n", homography_matrix)
    
    print(f"Applying perspective warp to size ({OUTPUT_WIDTH}, {OUTPUT_HEIGHT})...")
    # Warp the *undistorted* image
    final_rectified_img = cv2.warpPerspective(undistorted_img, homography_matrix, (OUTPUT_WIDTH, OUTPUT_HEIGHT))
    print("Perspective warp applied.")

    # Display comparison
    display_comparison('2. Undistorted Image', undistorted_img, '4. Final Rectified Image', final_rectified_img)
    cv2.imwrite(os.path.join(OUTPUT_DIR, '4_final_rectified.jpg'), final_rectified_img)

## 6. Conclusion

The process successfully loaded an image, corrected its lens distortion using pre-calculated calibration data, and then applied a perspective warp to obtain a rectified, top-down view of a specified region. This forms the foundation for many computer vision tasks where a consistent viewpoint is required, such as object measurement or recognition on a conveyor belt. The output images are saved in the `output_week1_notebook` directory.